In [36]:
# general
import os, sys, warnings, operator, itertools
warnings.filterwarnings('ignore')

# data structure
import pandas as pd
import numpy as np

# modeling and validate
import lightgbm as lgb
from sklearn.metrics import mean_squared_error as mse
def rmse(pred, true): return np.sqrt(mse(true, pred))

In [2]:
path = './dataset/origin/'
sample = pd.read_csv(path+'sample.csv')
train = pd.read_csv(path+'train.csv')
test = pd.read_csv(path+'test_mini.csv')

## eda

In [13]:
def obj_or_num(df, col):
    print(col)
    print('num of na : {}'.format(df[col].isna().sum()))
    if df[col].dtypes == 'O': print(df[col].value_counts(dropna=False))
    elif df[col].dtypes == 'int64' or df[col].dtypes == 'float64': 
        print(df[col].unique().shape)
        print(df[col].describe())
    else : print('is it proper type?')
    print('-'*50)
for df in (train, test):
    print('-'*20, 'new df', '-'*20)
    print('-'*50)
    for col in df.columns:
        obj_or_num(df, col)

-------------------- new df --------------------
--------------------------------------------------
country
num of na : 0
Colombia                  360
Malta                     360
Luxembourg                360
France                    360
Singapore                 360
Brazil                    360
United Kingdom            360
Israel                    360
Spain                     360
Greece                    360
Netherlands               360
Bulgaria                  360
Ireland                   360
Mexico                    360
United States             360
Austria                   360
Belgium                   360
Mauritius                 360
Italy                     360
Chile                     360
Japan                     360
Republic of Korea         360
Argentina                 360
Iceland                   360
Ecuador                   360
Puerto Rico               360
Costa Rica                360
Australia                 348
Norway                    348
Guatemal

In [3]:
sample.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101 entries, 0 to 100
Data columns (total 2 columns):
country    101 non-null object
y_pred     101 non-null float64
dtypes: float64(1), object(1)
memory usage: 1.7+ KB


In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26916 entries, 0 to 26915
Data columns (total 12 columns):
country              26916 non-null object
year                 26916 non-null int64
sex                  26916 non-null object
age                  26916 non-null object
suicides_no          26916 non-null int64
population           26916 non-null int64
suicides_100k_pop    26916 non-null float64
country_year         26916 non-null object
HDI_for_year         8364 non-null float64
gdp_for_year         26916 non-null object
gdp_per_capital      26916 non-null int64
generation           26916 non-null object
dtypes: float64(2), int64(4), object(6)
memory usage: 2.5+ MB


In [5]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 744 entries, 0 to 743
Data columns (total 11 columns):
country              744 non-null object
year                 744 non-null int64
sex                  744 non-null object
age                  744 non-null object
population           744 non-null int64
suicides_100k_pop    744 non-null float64
country_year         744 non-null object
HDI_for_year         0 non-null float64
gdp_for_year         744 non-null object
gdp_per_capital      744 non-null int64
generation           744 non-null object
dtypes: float64(2), int64(3), object(6)
memory usage: 64.0+ KB


### 타겟은 12개의 합인가?? <- 맞다

In [8]:
target = 'suicides_no'
train[target].head()

0    21
1    16
2    14
3     1
4     9
Name: suicides_no, dtype: int64

In [11]:
train.groupby('country')[target].agg('sum').reset_index().sort_values(by='country', ascending=True)

,country,suicides_no
0,Albania,1970
1,Antigua and Barbuda,10
2,Argentina,79146
3,Armenia,1764
4,Aruba,101
5,Australia,67084
6,Austria,47621
7,Azerbaijan,1656
8,Bahamas,93
9,Bahrain,463


In [9]:
sample.head()

,country,y_pred
0,Albania,35.977640
1,Antigua and Barbuda,8.352893
2,Argentina,10.157314
3,Armenia,27.624467
4,Aruba,69.397970


### 최신 날짜 없는 애들은 어떡하지??

In [16]:
countries = train['country'].value_counts(dropna=False)
cutOff = []
for country, val in zip(countries.index.tolist(), countries.values):
    if val == 12 : cutOff.append(country)

In [17]:
train.groupby('country')['year'].agg('max')

country
Albania                         2010
Antigua and Barbuda             2014
Argentina                       2014
Armenia                         2014
Aruba                           2011
Australia                       2014
Austria                         2014
Azerbaijan                      2007
Bahamas                         2013
Bahrain                         2014
Barbados                        2013
Belarus                         2014
Belgium                         2014
Belize                          2014
Bosnia and Herzegovina          2014
Brazil                          2014
Bulgaria                        2014
Cabo Verde                      2011
Canada                          2013
Chile                           2014
Colombia                        2014
Costa Rica                      2014
Croatia                         2014
Cuba                            2014
Cyprus                          2014
Czech Republic                  2014
Denmark                       

### train, test의 나라 포함관계

In [18]:
set(test['country'].unique()) == set(train['country'].unique())

False

In [19]:
print(set(test['country'].unique()) - set(train['country'].unique()))
print(set(train['country'].unique()) - set(test['country'].unique()))

set()
{'Montenegro', 'Macau', 'France', 'Trinidad and Tobago', 'Cabo Verde', 'Bahrain', 'Slovakia', 'Saint Kitts and Nevis', 'Aruba', 'El Salvador', 'Suriname', 'Azerbaijan', 'Barbados', 'Oman', 'Kiribati', 'Maldives', 'Paraguay', 'Portugal', 'Jamaica', 'Albania', 'Canada', 'Bahamas', 'Costa Rica', 'Belarus', 'New Zealand', 'Sri Lanka', 'Philippines', 'Guyana', 'Bosnia and Herzegovina', 'Dominica', 'Fiji', 'Saint Lucia', 'San Marino', 'Kuwait', 'Ireland', 'United Arab Emirates', 'Uzbekistan', 'Bulgaria'}


In [22]:
outOfTest = list(set(train['country'].unique()) - set(test['country'].unique()))
for c in cutOff:
     print(c in outOfTest)

True
True
True


### prep

[우습지만 세대 출처는 나무위키~](https://namu.wiki/w/%EC%84%B8%EB%8C%80#s-2.2)

- idea : gen + old 면 더 정확한 나이 추론할 수 있을 거 같은데?

#### 새로 만든 피처들

- year_2014
- sex_new
- age_new
- gdp_for_year_new
- generation_new

In [31]:
isMale = {'female':0,'male':1}
howOld = {'55-74 years': 4,'75+ years': 5,'15-24 years': 1,'5-14 years': 0,'35-54 years': 3,'25-34 years': 2,}
howGen = {'Generation X': 2, 'Silent': 4, 'Millenials': 1, 'Boomers': 3, 'G.I. Generation': 5, 'Generation Z': 0,}

train['year_2014'] = train['year'].apply(lambda x : 2014-x)
train['sex_new'] = train['sex'].map(isMale)
train['age_new'] = train['age'].map(howOld)
train['gdp_for_year_new'] = train['gdp_for_year'].apply(lambda x : int(x.replace(',', '')))
train['generation_new'] = train['generation'].map(howGen)

### 일단 완전한 데이터로만 빠르게 lgb

In [32]:
train_pure = train[train['country'].map(lambda x : x not in outOfTest)]
validate = train_pure[train_pure['year'].map(lambda x : x == 2014)]
train_pure = train_pure[train_pure['year'].map(lambda x : x != 2014)]

### 첫번째 테스트 : 12개 따로 구한 다음에 더하기

In [33]:
train_pure.columns

Index(['country', 'year', 'sex', 'age', 'suicides_no', 'population',
       'suicides_100k_pop', 'country_year', 'HDI_for_year', 'gdp_for_year',
       'gdp_per_capital', 'generation', 'year_2014', 'sex_new', 'age_new',
       'gdp_for_year_new', 'generation_new'],
      dtype='object')

In [ ]:
modelCols = ['population',
       'suicides_100k_pop', 'HDI_for_year',
       'gdp_per_capital', 'year_2014', 'sex_new', 'age_new',
       'gdp_for_year_new', 'generation_new']
x_train = train_pure[modelCols]
y_train = train_pure[target].values
x_validate = validate[modelCols]
y_validate = validate[target].values

lgb_train = lgb.Dataset(x_train, label = y_train)

param = {
    'num_iterations': 1000,
    'num_threads': 8,
}
import time
start = time.time()
bst = lgb.train(param, lgb_train)
print(time.time()-start)

In [37]:
pred = bst.predict(x_validate)
rmse(pred, y_validate)

55.44659298186641

In [56]:
y_validate

array([    0,  3231,    58,  2891,  1314,  1899,    24, 10631,  1789,
        2126,   722,  1453,    44,  1489,   629,   734,   241,   789,
         194, 10217,   565,     0,   512,  1925,    44,   382,  4147,
       24357,  3119,   461,   382,   930,    67,    32,   120,  6178,
        1839,   311,   548,   147,  5934,   220,    55, 13834,  2209,
       26541,     2,  1134,     4,   355,   388,   580,  3911,  1148,
        1032,  3952,  1617,   136,  7968,  4788, 42769,   591],
      dtype=int64)

In [40]:
pred
# 음수가 왜 나오는데....

array([-6.33885971e-02, -6.33885971e-02, -7.24219226e-03, -1.83521636e-01,
        1.22152439e-03, -1.01897010e-01,  1.27063502e-02,  1.27063502e-02,
        6.88527551e-02, -1.07426689e-01,  7.73164717e-02, -2.58020622e-02,
        1.91039426e+02,  7.29500656e+02,  5.10636318e+02,  4.19011485e+02,
        6.42894040e+02,  2.05904143e+02,  1.94133479e+02,  1.30000701e+02,
        1.01008521e+02,  2.37904976e+01,  3.68259413e+01,  3.19585117e+01,
        9.54390200e+00,  6.13465853e+00,  1.13713820e+01,  1.34363310e+01,
        6.46337391e+00,  2.38833910e+00,  2.92035286e+00,  2.61337913e+00,
        7.47424567e-01,  2.23673339e+00,  9.36409805e-01,  5.15136733e-01,
        8.99421265e+02,  1.71824521e+02,  3.56417583e+02,  4.58696645e+02,
        2.74684423e+02,  2.73896965e+02,  1.20499085e+02,  1.60519688e+02,
        9.34646458e+01,  4.55936991e+01,  1.17249946e+01,  8.67116794e+00,
        2.12485731e+02,  3.04533461e+02,  3.25765973e+02,  7.64779441e+01,
        9.96580596e+01,  

### 두번째 테스트 : agg한 채로 구하기

In [49]:
from scipy import stats
def mode(x):
    return stats.mode(x)[0][0]

def quantileRange(x):
    q1 = np.quantile(x, 0.25)
    q3 = np.quantile(x, 0.75)
    return q3 - q1

def aggregating(df):
    aggs = {
        'population': ['sum', 'min', 'max', 'mean', 'median', 'var', quantileRange],
       'suicides_100k_pop': ['sum', 'min', 'max', 'mean', 'median', 'var', quantileRange],
        'HDI_for_year': ['sum', 'min', 'max', 'mean', 'median', 'var', quantileRange],
       'gdp_per_capital': ['sum', 'min', 'max', 'mean', 'median', 'var', quantileRange],
        'year_2014': ['sum', 'max', 'mean', 'median', 'var', quantileRange,],
#         'sex_new': ['sum', 'min', 'max', 'mean', 'median', mode, 'var', quantileRange],
#         'age_new': ['sum', 'min', 'max', 'mean', 'median', mode, 'var', quantileRange],
       'gdp_for_year_new': ['sum', 'min', 'max', 'mean', 'median', 'var', quantileRange],
#         'generation_new': ['sum', 'min', 'max', 'mean', 'median', mode, 'var', quantileRange],
        'suicides_no' : ['sum'],
    }
    return df.groupby('country').agg(aggs).reset_index()

def renameCols(df):
    rename = []
    for col in df.columns.tolist():
        if col[0] == 'suicide_no' or col[1] == '': rename.append(col[0])
        else : rename.append(col[0]+'_'+col[1])
    df.columns = rename
    return df

train_pure_agg = aggregating(train_pure)
train_pure_agg = renameCols(train_pure_agg)
validate_agg = aggregating(validate)
validate_agg = renameCols(validate_agg)

modelCols = train_pure_agg.columns.tolist()
for col in ('country', 'suicides_no_sum'):
    modelCols.remove(col)
x_train = train_pure_agg[modelCols]
y_train = train_pure_agg['suicides_no_sum'].values
x_validate = validate_agg[modelCols]
y_validate = validate_agg['suicides_no_sum'].values

lgb_train = lgb.Dataset(x_train, label = y_train)

param = {
    'num_iterations': 1000,
    'num_threads': 8,
}

bst = lgb.train(param, lgb_train)

pred = bst.predict(x_validate)
rmse(pred, y_validate)

144342.92460583942

In [55]:
y_validate

array([    0,  3231,    58,  2891,  1314,  1899,    24, 10631,  1789,
        2126,   722,  1453,    44,  1489,   629,   734,   241,   789,
         194, 10217,   565,     0,   512,  1925,    44,   382,  4147,
       24357,  3119,   461,   382,   930,    67,    32,   120,  6178,
        1839,   311,   548,   147,  5934,   220,    55, 13834,  2209,
       26541,     2,  1134,     4,   355,   388,   580,  3911,  1148,
        1032,  3952,  1617,   136,  7968,  4788, 42769,   591],
      dtype=int64)

In [54]:
pred

array([  -3902.5305096 ,   -6126.56907395,   74170.53338424,
        293083.5041406 ,  143002.09571301,  180700.00092677,
         44112.91873331,  -11020.48413985,  -19330.10146548,
         51622.41984868,  -89164.26423292,  -89271.60319768,
        104770.51537081,  210507.5354363 ,  100162.99077919,
        -54608.1036356 ,  112604.94904365,  -73580.12471064,
         56786.49452154,  359406.38865064,  138488.89572083,
        -55975.6198518 ,   65025.81248109,   29722.5559578 ,
         98142.5655194 ,   88686.82546136,  276682.05517075,
        242437.69077277, -105750.84265489,    4590.84640838,
         81244.25351849,  -41338.02369505,  101063.76632406,
        105925.6274016 ,   57767.51790161,   53681.01867403,
        249183.66189183,    1559.39200572,  111922.59197809,
        -36801.26880383,  163376.70178288,  118756.41407606,
         79264.19795672,  219903.70663761,  150583.53480164,
        234775.16350231,   60357.92098197,  -58719.53566347,
        -47664.10243389,

## 결론 : 트리모델은 작동을 안합니당

- 아애 회귀적으로 풀어야 할 듯?!